In [0]:
dbutils.fs.ls("/mnt/silver/SalesLT/")

[FileInfo(path='dbfs:/mnt/silver/SalesLT/Address/', name='Address/', size=0, modificationTime=1747410302000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1747410317000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1747410320000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Product/', name='Product/', size=0, modificationTime=1747410324000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1747410327000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1747410330000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1747410333000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1747410336000),
 FileInf

In [0]:
dbutils.fs.ls("/mnt/gold/")

[FileInfo(path='dbfs:/mnt/gold/SalesLT/', name='SalesLT/', size=0, modificationTime=1747413616000)]

In [0]:
df=spark.read.format("delta").load("/mnt/silver/SalesLT/Address/")

In [0]:
from pyspark.sql.functions import col

def rename_columns_to_snake_case(df):
    """ 
    Convert column names from PascalCase to snake_case in a pyspark dataframe.
    Args:
        df(dataframe): the input dataframe with columns to be renamed. 
    Returns:
        A new dataframe with new columns converted to snake_case.
    """
    #Get the list of Columns
    column_names=df.columns

    #Dictionary to hold old and new column name mappings
    rename_map={}
    
    for old_col in column_names:
        new_col="".join([
            "_"+char.lower() if (
                char.isupper() and 
                idx>0 and       #ensure it is not first character
                not old_col[idx-1].isupper() #ensure the previous character is not uppercase
                ) else char.lower() 
            for idx, char in enumerate(old_col)
        ]).lstrip("_") # remove any leading underscore 

        #Avoid renaming to an existing column name 
        if new_col in rename_map.values():
            raise ValueError(f"duplicate column name found after renaming: '{new_col}'")
        #map the old column name to new column name
        rename_map[old_col]=new_col
    #rename columns using the mapping 
    for old_col,new_col in rename_map.items():
        df=df.withColumnRenamed(old_col,new_col)

    return df

#Example usage
# df= rename_columns_to_snake_case(df)            

In [0]:
df= rename_columns_to_snake_case(df)

All table columns transformations
****

In [0]:
table_temp=[]

for i in dbutils.fs.ls("/mnt/silver/SalesLT/"):
    table_temp.append(i)

In [0]:
table_temp

[FileInfo(path='dbfs:/mnt/silver/SalesLT/Address/', name='Address/', size=0, modificationTime=1747410302000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1747410317000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1747410320000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/Product/', name='Product/', size=0, modificationTime=1747410324000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1747410327000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1747410330000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModel/', name='ProductModel/', size=0, modificationTime=1747410333000),
 FileInfo(path='dbfs:/mnt/silver/SalesLT/ProductModelProductDescription/', name='ProductModelProductDescription/', size=0, modificationTime=1747410336000),
 FileInf

In [0]:
table_name=[]
for i in dbutils.fs.ls("mnt/silver/SalesLT"):
    table_name.append(i.name.split('/')[0])

In [0]:
table_name

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

In [0]:
for name in table_name:
    path='/mnt/silver/SalesLT/'+name
    print(path)
    df=spark.read.format("delta").load(path)

    df=rename_columns_to_snake_case(df)

    output_path='/mnt/gold/SalesLT/'+name+'/'
    df.write.mode("overwrite").format("delta").save(output_path)



/mnt/silver/SalesLT/Address
/mnt/silver/SalesLT/Customer
/mnt/silver/SalesLT/CustomerAddress
/mnt/silver/SalesLT/Product
/mnt/silver/SalesLT/ProductCategory
/mnt/silver/SalesLT/ProductDescription
/mnt/silver/SalesLT/ProductModel
/mnt/silver/SalesLT/ProductModelProductDescription
/mnt/silver/SalesLT/SalesOrderDetail
/mnt/silver/SalesLT/SalesOrderHeader
